<a href="https://it-omscholing.nl/locaties/hogeschool-rotterdam/">
<div>
<a><img src='pics/banner.PNG'/></a>
</div>
<div>
<a href=''><img src='pics/miw.PNG'/></a>
<em>Author: Jeroen Boogaard</em>
</div>
</a>

<h1>Practicum SQL met <a href="https://geopandas.org/en/stable/index.html">GeoPandas</a></h1>
<img src="pics/geopandas_icon.png" width="25%" height="25%"/>

<h2>Environment Variables</h2>

In [ ]:
import os
os.environ['export PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] ='notebook'
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'

In [ ]:
!(pip list | grep geopandas) || pip install geopandas

<h2>Imports<h2>

In [ ]:
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import pyspark.sql
import pandas as pd
import pyspark.pandas as ps
import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot
import numpy as np

<h2>Data Visualization</h2>

<h3>Extract</h3>

In [ ]:
data_dir = "../geo/ref-nuts-2021-01m/"

euPath = data_dir + "NUTS_RG_01M_2021_3035_LEVL_1.json"
gdf = gpd.read_file(euPath)

<h3>Transform</h3>

In [ ]:
gdf.crs = "EPSG:3035"
gdfBen = gdf[(gdf.CNTR_CODE == "BE") | (gdf.CNTR_CODE == "NL") | (gdf.CNTR_CODE == "LU")]
gdfCrsBen = gdfBen.to_crs("EPSG:3857")
gdfNl = gdf[gdf.CNTR_CODE == "NL"]
gdfCrsNl = gdfNl.to_crs("EPSG:3857")

In [ ]:
ax = gdfCrsBen.plot(figsize=(20,15), color="lightgray")

# Grenzen Benelux landen
gdfCrsBen.boundary.plot(color="darkgreen", ax=ax)

# Provincies Nederland
gdfCrsNl.boundary.plot(color="red", ax=ax)

# add background map by OpenStreetMap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.HOT)

**Ingezoomed op de Benelux en Nederland gemarkeerd**

<h3>Transform</h3>

<h2>PySpark</h2>

Load

In [ ]:
eurCsvFile = "../csv/countries_europe.csv"
euCsvFile = "../csv/european_union.csv"
eurCodesCsvFile = "../csv/country-codes.csv"

In [ ]:
# Create a Spark Session
spark = SparkSession.builder.appName('SQL').getOrCreate()

In [ ]:
# Create Spark dataframe from csvfile
eurDf = spark.read.csv(eurCsvFile, inferSchema=True, header=True)
eurDf.printSchema()

In [ ]:
# Create a view
eurDf.createOrReplaceTempView('countries')

**query op het dataframe via <i>methods</i>**

In [ ]:
eurDf.select("Code").show(10)

**<i>SQL</i> query op het dataframe**

In [ ]:
countriesQuery = """
    SELECT Code
    FROM countries
"""

eurCodes = spark.sql(countriesQuery)
eurCodes.show(10)

In [ ]:
eurDf.select("Code").show(10)

<h3>Exercise 1</h3>
<p>
Maak op dezelfde manier dataframes euDf en eurCodesDf aan door het importeren vanuit de csv-files
</p>

In [ ]:
euDf = spark.read.csv(euCsvFile, inferSchema=True, header=True)

In [ ]:
euDf.show()

<h3>Exercise 2a</h3>
<p>
    Maak een <u>koppeling</u> tussen <u>euDf</u> en <u>eurCodesDf</u> via kolom <u>Code</u> en <u>ISO3166-1-Alpha-3</u> 
</p>

In [ ]:
# Oplossing

<h3>Exercise 2b</h3>

<h4>Gegeven</h4>

<p>De dataset gdf bevat tweeletterige landcodes die je ook kunt terugvinden in de dataset <u>eurCodesDf</u></>

In [ ]:
gdf.CNTR_CODE.tolist()[:10]

<h4>Gevraagd</h4>
<p>
    Maak een geopandas dataframe <u>gdfEu</u> gebaseerd op <u>gdf</u> en markeer hierin de EU landen met <span style="color:red">rood</span>.
    <strong>Hint</strong>Gebruik de koppeling uit 2a voor het verkrijgen van de tweeletterige landcodes van EU landen en filter daarmee het geodataframe.
</p>

<h4>Oplossing</h4>

In [ ]:
# Oplossing

**References**
<ul>
    <li>        
        <a href="https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53">Pyspark SQL basics</a>       
    </li>
    <li>
        <a href="https://ec.europa.eu/eurostat/web/main/home">Eurostat</a>
    </li>
</ul>    